In [6]:
from game_logic.ghostAgents import RandomGhost
from entregables.maxNAgent import MaxNAgent
from game_logic.randomPacman import RandomPacman
from game_logic.PacmanEnvAbs import PacmanEnvAbs
import random
import math
import numpy as np
from game_logic.game_util import process_state
from main import get_default_agents
import timeit 
import pandas as pd

all_layouts = [
        "custom1",
        "custom2",
        "capsuleClassic",
        "contestClassic",
        "mediumClassic",
        "minimaxClassic",
        "openClassic",
        "originalClassic",
        "smallClassic",
        "testClassic",
        "trappedClassic",
        "trickyClassic",
        "mediumGrid",
        "smallGrid"
    ]
    
class TestParams():
    def __init__(self, test_name, layout, pacman_agent, ghost_agent_0, ghost_agent_1):        
        self.test_name = test_name
        self.layout = layout
        self.pacman_agent = pacman_agent
        self.ghost_agent_0 = ghost_agent_0
        self.ghost_agent_1 = ghost_agent_1


In [12]:
def run_test(test_params):  
    t0 = timeit.default_timer()   
    pacman_agent = test_params.pacman_agent
    ghost_agent_0 = test_params.ghost_agent_0
    ghost_agent_1 = test_params.ghost_agent_1
    agents = [pacman_agent, ghost_agent_0, ghost_agent_1]
    agents.extend(get_default_agents(3, 10))    
    done = False
    env = PacmanEnvAbs(agents = agents, view_distance = (2, 2))      
    game_state = env.reset(enable_render= False, layout_name= test_params.layout)
    turn_index = 0
    while (not(done)):    
        action = agents[turn_index].getAction(game_state)
        game_state, rewards, done, info = env.step(action, turn_index)        
        turn_index = (turn_index + 1) % env._get_num_agents()  
    t1 = timeit.default_timer()    
    time = t1-t0  
    assert(game_state.getNumAgents()>=2)
    if game_state.getNumAgents()==2:
        ret = game_state.get_rewards()
        ret.append(np.nan)
    else:
        ret = game_state.get_rewards()[0:3]
    return ret, time

In [13]:
pacman_agent = RandomPacman(index=0)
ghost_agent_0 = MaxNAgent(index=1, unroll_type="MCTS", max_unroll_depth=12, number_of_unrolls=6)
ghost_agent_1 = RandomGhost(index=2)
sample_test = TestParams("PrimerTest", "mediumGrid", pacman_agent, ghost_agent_0, ghost_agent_1)
print(run_test(sample_test))

([-1495, -5619.0, nan], 1.6275760519783944)


In [14]:
selected_layouts = [  
        "originalClassic",  
        "mediumGrid",
        "smallGrid"
    ]
pacmans = [("rnd_pcmn", RandomPacman(index = 0)), ("MAXN_pcmn", MaxNAgent(index = 0, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10))]
ghosts0s = [
    ("MAXN_MC_gh0", MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh0", MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]
ghosts1s = [
    ("MAXN_MC_gh1", MaxNAgent(index = 2, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh1", MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]

In [15]:
all_tests = []
for layout in selected_layouts:
    for pacman_name, pacman_agent in pacmans:
        for ghost0_name, ghost_0 in ghosts0s:
            for ghost1_name, ghost_1 in ghosts1s:
                name = layout + "_" + pacman_name + "_" + ghost0_name + "_" + ghost1_name
                test = TestParams(name, layout, pacman_agent, ghost_agent_0, ghost_agent_1)
                all_tests.append(test)
number_of_runs = 10

In [16]:
results = []
t0 = timeit.default_timer()   
for i in range(len(all_tests)):   
        values = []
        times = []    
        for _ in range(number_of_runs):
            value, time = run_test(all_tests[i])            
            values.append(value)
            times.append(time)
        # compute avg and std and and add to results
        value_avg = np.round(np.mean(np.array(values), axis=0), 2)
        time_avg = np.round(np.mean(np.array(times)), 2)
        value_std = np.round(np.std(np.array(values), axis=0), 2)
        time_std = np.round(np.std(np.array(times)), 2)
        result = {
            'TestName': all_tests[i].test_name, 
            'Time_Avg': time_avg, 
            'Time_Std': time_std, 
            'P_Value_Avg': value_avg[0], 
            'P_Value_Std': value_std[0],
            'GH0_Value_Avg': value_avg[1],
            'GH0_Value_Std': value_std[1],
            'GH1_Value_Avg': value_avg[2], 
            'GH1_Value_Std': value_std[2]
          }
        results.append(result)
results = pd.DataFrame(results)
t1 = timeit.default_timer()  
time = t1 - t0
print("All tests time: ", time, "seconds.")
display(results)

All tests time:  2902.569713953999 seconds.


,TestName,Time_Avg,Time_Std,P_Value_Avg,P_Value_Std,GH0_Value_Avg,GH0_Value_Std,GH1_Value_Avg,GH1_Value_Std
0,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,13.01,5.30,-1453.7,50.76,-2002.6,973.06,-1839.8,812.10
1,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS...,17.88,8.82,-1431.2,50.76,-2817.4,1442.84,-2663.1,1519.53
2,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,14.41,10.72,-1420.5,46.25,-2321.5,1908.58,-2158.6,2178.74
3,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,17.91,8.84,-1471.4,39.86,-2844.6,1476.00,-2998.4,1661.30
4,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,54.13,34.27,-1483.1,69.55,-8135.1,5319.50,-25085.2,16210.24
5,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MCT...,50.99,28.61,-1459.9,42.38,-7677.9,4329.56,-23730.8,13633.17
6,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,45.80,24.59,-1446.4,37.14,-7231.6,3949.45,-21023.1,11753.98
7,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,50.71,16.84,-1447.4,32.96,-7978.1,2936.30,-24042.9,8412.44
8,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,1.44,0.71,-1507.0,8.21,-3996.2,2703.37,NaN,NaN
9,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS_gh1,1.45,0.49,-1502.2,5.46,-3504.2,1543.90,NaN,NaN
